In [1]:
import pandas as pd 
import numpy as np
import dask.dataframe as dd

# parallelize the work using dask
from dask.distributed import Client
client = Client()

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 3.74 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45727,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 3.74 GiB
Comm: tcp://127.0.0.1:33043,Total threads: 2
Dashboard: http://127.0.0.1:46839/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:33477,


In [3]:
#df_demo = dd.read_csv(r"./demo_Z0.00014/output_*.csv", dtype={'BEvent': float})
df0 = dd.read_csv("./Z0.02_a3_cM70/output_0.csv", dtype={'BEvent': float})

In [4]:
df0

,ID,name,Mass_0,Radius_0,Phase_0,PhaseBSE_0,RemnantType_0,Hsup_0,Mass_1,Radius_1,Phase_1,PhaseBSE_1,RemnantType_1,Hsup_1,Semimajor,Eccentricity,BWorldtime,Period,GWtime,RL0,RL1,BEvent
npartitions=1152,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float64,float64,int64,float64,int64,float64,float64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Wolf Rayet - Black Hole systems

In [5]:
df0_WRBH = df0.loc[( ((df0.PhaseBSE_0==8) | (df0.PhaseBSE_0==7)) & (df0.PhaseBSE_1==14.0) ) | ( ((df0.PhaseBSE_1==8) | (df0.PhaseBSE_1==7)) & (df0.PhaseBSE_0==14.0) )]
data_WRBH = df0_WRBH[['ID', 'Mass_0', 'Mass_1', 'Semimajor', 'Eccentricity']].drop_duplicates('ID').compute()
id0_WRBH = data_WRBH['ID']

2024-05-15 18:25:10,763 - distributed.worker.memory - WARNING - gc.collect() took 2.196s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2024-05-15 18:25:10,765 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 770.05 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:25:10,837 - distributed.worker.memory - WARNING - Worker is at 52% memory usage. Resuming worker. Process memory: 501.37 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:26:08,180 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 618.03 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:26:13,775 - distributed.worker.memory - WARNING - Unmanage

### GW BBH merge

In [6]:
id0_GW = df0.loc[((df0.PhaseBSE_0 == 14.0) & (df0.PhaseBSE_1 == 14.0) & ((df0.GWtime + df0.BWorldtime) < int(14e+03)))]['ID'].drop_duplicates().compute()

2024-05-15 18:37:37,773 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 683.03 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:37:53,467 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 666.03 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:38:21,276 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

In [7]:
id0_both = np.intersect1d(id0_WRBH, id0_GW)

In [8]:
print('Binary systems of a Wolf-Rayet and a Black Hole: ', len(id0_WRBH))
print('Binary systems that merge via GW: ', len(id0_GW))
print('Binary systems that evolve from a WR-BH and merge via GW: ', len(id0_both))

Binary systems of a Wolf-Rayet and a Black Hole:  130479
Binary systems that merge via GW:  340
Binary systems that evolve from a WR-BH and merge via GW:  306


### Mass transfer events

In [9]:
df0_WRBH_stable = df0_WRBH.loc[df0_WRBH['BEvent']==5, ['ID','BEvent']].groupby('ID').count()

In [10]:
df0_WRBH_unstable = df0_WRBH.loc[(df0_WRBH['BEvent']==7) | (df0_WRBH['BEvent']==11), ['ID','BEvent']].groupby('ID').count()

In [11]:
mass_transfer0 = df0_WRBH_stable.merge(df0_WRBH_unstable, how='outer', on='ID').rename(columns={'BEvent_x': 'MTEvents_stable', 'BEvent_y': 'MTEvents_unstable'})
mass_transfer0['MTEvents_unstable'] = mass_transfer0['MTEvents_unstable'].fillna(0).astype(int)
mass_transfer0['MTEvents_stable'] = mass_transfer0['MTEvents_stable'].fillna(0).astype(int)

data_0 = mass_transfer0.merge(data_WRBH, how='outer', on='ID')
data_0.compute()

2024-05-15 18:47:40,941 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 675.41 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:47:40,920 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 676.45 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:47:41,446 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2024-05-15 18:47:47,600 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 03968c9ff7f80c9580f5e7d9d4e05239 initialized by task ('hash-join-transfer-03968c9ff7f80c9580f5e7d9d4e05239', 0) executed on worker tcp://127.0.0.1:33043
2024-05-15 18:57:53,029 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 0543e901a305484294131f58bdeb1c22 initialized by task ('hash-join-transfer-0543e901a305484294131f58bdeb1c22', 0) executed on worker tcp://127.0.0.1:46813
2024-05-15 18:57:53,113 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 8dc1925f8b4f5e7c08fe312bb95864f3 initialized by task ('hash-join-transfer-8dc1925f8b4f5e7c08fe312bb95864f3', 0) executed on worker tcp://127.0.0.1:34319
2024-05-15 18:57:53,925 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 0543e901a305484294131f58bdeb1c22 deactivated due to stimulus 'task-finished-1715792273.9198132'
2024-05-15 18:57:53,936 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 8dc1925f8b4f5e7c08fe312bb

,ID,MTEvents_stable,MTEvents_unstable,Mass_0,Mass_1,Semimajor,Eccentricity
0,7,NaN,NaN,18.225050,19.32260,NaN,NaN
1,11,NaN,NaN,21.816500,22.43766,2488.11,0.755503
2,33,NaN,NaN,1.531003,15.52529,NaN,NaN
3,115,NaN,NaN,20.162420,23.16621,NaN,NaN
4,129,NaN,NaN,19.957810,21.45724,NaN,NaN
...,...,...,...,...,...,...,...
130474,4180109,NaN,NaN,1.313853,14.98446,NaN,NaN
130475,4180149,NaN,NaN,22.531700,26.95646,18173.91,0.952452
130476,4180153,NaN,NaN,18.979820,22.57124,NaN,NaN
130477,4180212,NaN,NaN,12.622030,0.75723,NaN,NaN


In [12]:
data_0.to_csv('out.csv', index=False)  

2024-05-15 18:59:31,056 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 670.97 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:59:35,012 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 672.61 MiB -- Worker memory limit: 0.93 GiB
2024-05-15 18:59:35,606 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

['/home/gaia/LCP-B/project/out.csv/0.part']